# Query FX Rates from ECB APIs
Copyright (C) 2021 OS-Climate

This sample shows:
* How to connect to the Kafka cluster streaming Daily FX Rates from ECB APIs
* How to query and retrieve the rates based on currency and date parameters

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas for data processing
%pip install pandas

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools

In [2]:
# Report exceptions only in cells output
%xmode Minimal

Exception reporting mode: Minimal


In [3]:
from dotenv import dotenv_values, load_dotenv
import os
import pandas as pd
import trino
from sqlalchemy.engine import create_engine
import osc_ingest_trino as osc

Environment and Execution Variables

In [4]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

Connect to Trino and load the data

In [5]:
# Define execution variables for querying rates in Trino
query_catalog = 'kafka_fx'
query_schema = 'ecb-fx'
query_table = 'fx'

In [6]:
engine = osc.attach_trino_engine(verbose=True, catalog=query_catalog)

using connect string: trino://caldeirav@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/kafka_fx


In [7]:
sql=f"""
select currency, \"currency-denom\", \"end-time-period\", \"obs-value\"
from {query_catalog}.\"{query_schema}\".{query_table}
where date(\"end-time-period\") between date('2022-10-01') and date('2022-10-22')
"""

In [8]:
df = pd.read_sql(sql, engine).convert_dtypes()

In [9]:
print(df.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   currency         44 non-null     string 
 1   currency-denom   44 non-null     string 
 2   end-time-period  44 non-null     string 
 3   obs-value        44 non-null     Float64
dtypes: Float64(1), string(3)
memory usage: 1.5 KB
None


In [10]:
df.head()

,currency,currency-denom,end-time-period,obs-value
0,TRY,EUR,2022-10-03 21:59:59.999,18.124
1,TRY,EUR,2022-10-04 21:59:59.999,18.3374
2,TRY,EUR,2022-10-05 21:59:59.999,18.4201
3,TRY,EUR,2022-10-06 21:59:59.999,18.3191
4,THB,EUR,2022-10-03 21:59:59.999,37.181
